<a href="https://colab.research.google.com/github/ravishastri/demo/blob/master/Funetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset


model_name='distilgpt2'


In [9]:
model=GPT2LMHeadModel.from_pretrained(model_name)
tokenizer=GPT2Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
tokenizer.pad_token=tokenizer.eos_token

In [11]:
dataset=load_dataset("wikitext","wikitext-2-raw-v1")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [12]:
small_dataset=dataset['train'].select(range(200))

In [13]:
def tokenize_function(examples):
  encodings=tokenizer(examples['text'], padding="max_length",truncation=True, max_length=64)
  encodings['labels']=encodings['input_ids'].copy()
  return encodings

tokenized_dataset=small_dataset.map(tokenized_function, batched=True)
tokenized_dataset=tokenized_dataset.shufflE(seed=42)

In [16]:
tokenized_dataset=small_dataset.map(tokenize_function, batched=True)
tokenized_dataset=tokenized_dataset.shuffle(seed=42)

In [17]:
training_args = TrainingArguments(
    output_dir="./distilgpt2-finetuned-demo", # directory where the model would be saved
    overwrite_output_dir=True,
    num_train_epochs=1, #1 epoch-> pass over the entire dataset once
    per_device_train_batch_size=2,
    save_steps=50, #50 steps -> save the model checkpoint
    save_total_limit=1, # at any point you have the recent checkpoint
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # Avoid reporting to any service
)

In [18]:
trainer = Trainer(
    model=model, # the model that we want to fine tune
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

<ipython-input-18-57193a53dfde>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
# Train the model
trainer.train() #trainable parameters

# Save the fine-tuned model
trainer.save_model("./distilgpt2-finetuned-demo")
tokenizer.save_pretrained("./distilgpt2-finetuned-demo")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.441400
20,1.949100
30,1.525400
40,2.353000
50,2.388600
60,1.945200
70,1.546000
80,2.031000
90,2.004200
100,1.178900


('./distilgpt2-finetuned-demo/tokenizer_config.json',
 './distilgpt2-finetuned-demo/special_tokens_map.json',
 './distilgpt2-finetuned-demo/vocab.json',
 './distilgpt2-finetuned-demo/merges.txt',
 './distilgpt2-finetuned-demo/added_tokens.json')